In [1]:
import platform
import tensorflow
import keras
print("Platform: {}".format(platform.platform()))
print("Tensorflow version: {}".format(tensorflow.__version__))
print("Keras version: {}".format(keras.__version__))

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from IPython.display import Image

/home/dotrungkien/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Platform: Linux-4.4.0-112-generic-x86_64-with-Ubuntu-16.04-xenial
Tensorflow version: 1.4.0
Keras version: 2.0.9


In [18]:
train_dir = "train"
validation_dir = "validation"
test_dir = "test"
chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
n_classes = 36
n_len = 7
height, width = 224, 224

In [19]:
from glob import glob
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [39]:
def read_data(img_dir):
    img_paths = glob("{}/*.png".format(img_dir))
    images = []
    labels = []
    for img_path in img_paths:
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
#         image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)/255.
        label = [chars.index(i) for i in img_path.split('/')[-1].split('_')[1]]
        images.append(image)
        labels.append(label)
    return np.array(images), np.array(labels)

def list_to_categorical(y):
    y_cat = []
    for i in y:
        j = np.zeros((7, n_classes))
        j[np.arange(7), i] = 1
        y_cat.append(j)
    return np.array(y_cat)

In [40]:
x_train, y_train = read_data(train_dir)
y_train = list_to_categorical(y_train)
x_train.shape, y_train.shape

((1000, 224, 224, 3), (1000, 7, 36))

In [41]:
x_val, y_val = read_data(validation_dir)
y_val = list_to_categorical(y_val)
x_val.shape, y_val.shape

((500, 224, 224, 3), (500, 7, 36))

In [42]:
x_test, y_test = read_data(test_dir)
y_test = list_to_categorical(y_test)
x_test.shape, y_test.shape

((500, 224, 224, 3), (500, 7, 36))

In [49]:
def gen_batch(batch_size=32):
    while True:
        random_indices = np.random.choice(x_train.shape[0], batch_size)
        x_batch = x_train[random_indices]
        y_batch = y_train[random_indices].reshape(-1, batch_size, n_classes)
        yield x_batch, list(y_batch)

In [50]:
x_batch, y_batch = next(gen_batch())
type(x_batch), type(y_batch)

(numpy.ndarray, list)

In [55]:
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = Flatten()(x)
x = Dropout(0.25)(x)
x = [Dense(n_classes, activation='softmax', name="char_{}".format(i))(x) for i in range(n_len)]
model = Model(input=base_model.input, output=x)

/home/dotrungkien/py35/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  # Remove the CWD from sys.path while we load stuff.


In [56]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_8[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [57]:
# conv_base.trainable = False
for layer in base_model.layers:
    layer.trainable = False

In [58]:
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(),
             metrics=['acc'])

# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

# datagen.fit(x_train)

history = model.fit_generator(gen_batch(),
                              epochs=10, 
                              steps_per_epoch=len(x_train)//32,
                              validation_data=gen_batch(), 
                              validation_steps=5, 
                              verbose=2)

Epoch 1/10
 - 6s - loss: 34.6911 - char_0_loss: 4.9295 - char_1_loss: 4.8494 - char_2_loss: 5.0364 - char_3_loss: 4.9879 - char_4_loss: 4.9053 - char_5_loss: 4.9032 - char_6_loss: 5.0794 - char_0_acc: 0.0282 - char_1_acc: 0.0302 - char_2_acc: 0.0282 - char_3_acc: 0.0181 - char_4_acc: 0.0323 - char_5_acc: 0.0252 - char_6_acc: 0.0323 - val_loss: 33.9358 - val_char_0_loss: 4.6869 - val_char_1_loss: 5.0030 - val_char_2_loss: 4.6456 - val_char_3_loss: 5.1253 - val_char_4_loss: 4.4589 - val_char_5_loss: 4.7060 - val_char_6_loss: 5.3101 - val_char_0_acc: 0.0250 - val_char_1_acc: 0.0250 - val_char_2_acc: 0.0375 - val_char_3_acc: 0.0187 - val_char_4_acc: 0.0125 - val_char_5_acc: 0.0312 - val_char_6_acc: 0.0250
Epoch 2/10
 - 5s - loss: 34.3020 - char_0_loss: 4.7294 - char_1_loss: 5.0052 - char_2_loss: 4.8231 - char_3_loss: 4.8911 - char_4_loss: 4.8878 - char_5_loss: 4.9201 - char_6_loss: 5.0454 - char_0_acc: 0.0393 - char_1_acc: 0.0202 - char_2_acc: 0.0312 - char_3_acc: 0.0272 - char_4_acc: 0.04

In [59]:
from keras.models import load_model
model = load_model('model.h5')
model.summary()

/home/dotrungkien/py35/lib/python3.5/site-packages/keras/engine/topology.py:1253: UserWarning: Update your `InputLayer` call to the Keras 2 API: `InputLayer(sparse=False, name="input_1", batch_input_shape=[None, 170..., dtype="float32")`
  return cls(**config)
/home/dotrungkien/py35/lib/python3.5/site-packages/keras/engine/topology.py:1253: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_initializer="glorot_uniform", name="convolution2d_1", trainable=True, kernel_regularizer=None, padding="valid", filters=32, kernel_size=(3, 3), bias_regularizer=None, activation="relu", kernel_constraint=None, data_format="channels_last", activity_regularizer=None, bias_constraint=None, strides=[1, 1], use_bias=True)`
  return cls(**config)
/home/dotrungkien/py35/lib/python3.5/site-packages/keras/engine/topology.py:1253: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_initializer="glorot_uniform", name="convolution2d_2", trainable=True, kernel_regular

ValueError: Unknown loss function:<lambda>

In [60]:
import keras
keras.__version__


'2.0.9'